In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    TrainingArguments, Trainer,
    DataCollatorForSeq2Seq
)

In [1]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.3 MB/s eta 0:00:00


In [3]:
# 1. Load datasets
def load_dataset(file_path):
    df = pd.read_csv(file_path)
    df = df.dropna()
    df = df[["question", "answer"]]
    return Dataset.from_pandas(df)

train_dataset = load_dataset("train_data.csv")
val_dataset = load_dataset("val_data.csv")
test_dataset = load_dataset("test_data.csv")

In [ ]:


model_checkpoint = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)



tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:

max_input_length = 128
max_target_length = 128

def preprocess(examples):
    inputs = ["question: " + q for q in examples["question"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")

    labels = tokenizer(examples["answer"], max_length=max_target_length, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(preprocess, batched=True)
val_dataset = val_dataset.map(preprocess, batched=True)
test_dataset = test_dataset.map(preprocess, batched=True)

Map:   0%|          | 0/12047 [00:00<?, ? examples/s]

Map:   0%|          | 0/2581 [00:00<?, ? examples/s]

Map:   0%|          | 0/2582 [00:00<?, ? examples/s]

In [ ]:

training_args = TrainingArguments(
    output_dir="./finetuned_fingpt_qa_bot",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=1,
    logging_dir="./logs",
    logging_steps=10,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [ ]:


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


<ipython-input-9-6f1b2d2435cc>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: williammokonemafube (williammokonemafube-code) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,3.204600,2.905564
2,3.012900,2.871563
3,3.129700,2.857692
4,2.885300,2.850880
5,2.918600,2.848738


TrainOutput(global_step=7530, training_loss=3.1047016941693673, metrics={'train_runtime': 5088.8921, 'train_samples_per_second': 11.837, 'train_steps_per_second': 1.48, 'total_flos': 1.031158978117632e+16, 'train_loss': 3.1047016941693673, 'epoch': 5.0})

In [ ]:
trainer.save_model("./finetuned_fingpt_qa_bot")
tokenizer.save_pretrained("./finetuned_fingpt_qa_bot")

('./finetuned_fingpt_qa_bot/tokenizer_config.json',
 './finetuned_fingpt_qa_bot/special_tokens_map.json',
 './finetuned_fingpt_qa_bot/spiece.model',
 './finetuned_fingpt_qa_bot/added_tokens.json',
 './finetuned_fingpt_qa_bot/tokenizer.json')

In [ ]:

test_results = trainer.evaluate(eval_dataset=test_dataset)
print("\n Test Set Evaluation Results:")
for key, value in test_results.items():
    print(f"{key}: {value:.4f}")



📊 Test Set Evaluation Results:
eval_loss: 2.8357
eval_runtime: 55.1269
eval_samples_per_second: 46.8370
eval_steps_per_second: 5.8590
epoch: 5.0000


In [ ]:

def generate_predictions(dataset):
    model.eval()
    predictions = []
    inputs = ["question: " + q for q in dataset["question"]]

    for i in range(0, len(inputs), 8):  # batching
        batch_inputs = inputs[i:i+8]
        tokenized_inputs = tokenizer(batch_inputs, return_tensors="pt", padding=True, truncation=True, max_length=128).to(model.device)

        with torch.no_grad():
            outputs = model.generate(**tokenized_inputs, max_length=128)
            decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            predictions.extend(decoded)

    return predictions



In [15]:
import torch

pred_answers = generate_predictions(test_dataset)


In [ ]:

print("\n Sample Predictions vs Actual Answers:\n")
for i in range(min(10, len(pred_answers))):
    print(f"Q{i+1}: {test_dataset[i]['question']}")
    print(f"Predicted: {pred_answers[i]}")
    print(f"Actual   : {test_dataset[i]['answer']}\n")



📋 Sample Predictions vs Actual Answers:

Q1: Company revenue increased however stock price did not
Predicted: The stock price is a measure of the companys performance. The companys revenue is a measure of the companys performance. The companys revenue is a measure of the companys performance. The companys revenue is a measure of the companys performance. The companys revenue is a measure of the companys performance. The companys revenue is a measure of the companys performance. The companys revenue is a measure of the companys performance. The companys revenue is a measure of the companys performance. The companys revenue is a measure of the companys performance. The company
Actual   : The company released its 2nd Quarter Revenue of 1,957,921 a couple days ago however the stock did not move up in any way. Why? If the company is making money shouldnt the stock go up. But that result doesnt indicate that the company is making money. The word for making money is profit, not revenue. Prof